In [1]:
#from statsmodels.tsa.holtwinters import ExponentialSmoothing
import xgboost as xgb
import pandas as pd
import json
from datetime import datetime
from tabulate import tabulate

df = pd.read_csv('btc_historical.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [2]:
def print_df(df):
    print(tabulate(df, headers='keys', tablefmt='psql'))
def create_time_features(df):
    if 'Date' not in df.columns:
        print('requires date column')
        return
    df = df.copy()
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['DOM'] = df['Date'].dt.day
    df['DOW'] = df['Date'].dt.day_name()
    return df

def create_market_features(df):
    df = df.copy()
    df['Gain'] = (df['Close'] > df['Open']).astype(int)
    df['Lagged_Gain'] = df['Gain'].shift(1)
    df['Rolling_Gain'] = df['Gain'].rolling(7).mean()
    df['Difference'] = df['Close'] - df['Open']
    df['Lagged_Difference'] = df['Difference'].shift(1)
    df['Rolling_Difference'] = df['Difference'].rolling(7).mean()
    df['Lagged_Volume'] = df['Volume'].shift(1)
    df['Range'] = df['High'] - df['Low']
    df['Lagged_Range'] = df['Range'].shift(1)
    return df

In [3]:
df = create_time_features(df)
df = create_market_features(df)
print_df(df)

+------+---------------------+-----------+-----------+-----------+-----------+-------------+--------------+--------+---------+-------+-----------+--------+---------------+----------------+--------------+---------------------+----------------------+-----------------+-------------+----------------+
|      | Date                |      Open |      High |       Low |     Close |   Adj Close |       Volume |   Year |   Month |   DOM | DOW       |   Gain |   Lagged_Gain |   Rolling_Gain |   Difference |   Lagged_Difference |   Rolling_Difference |   Lagged_Volume |       Range |   Lagged_Range |
|------+---------------------+-----------+-----------+-----------+-----------+-------------+--------------+--------+---------+-------+-----------+--------+---------------+----------------+--------------+---------------------+----------------------+-----------------+-------------+----------------|
|    0 | 2015-01-01 00:00:00 |   320.435 |   320.435 |   314.003 |   314.249 |     314.249 |      8036550 

In [4]:
FEATURES = ['Year', 'Month', 'DOM', 'DOW', 'Lagged_Gain', 'Rolling_Gain', 'Lagged_Difference', 'Rolling_Difference', 'Lagged_Volume', 'Lagged_Range']
TARGET =  ['Gain']

# split into test/train
split_date = '2024-06-28'
train_df = df[df['Date'] < split_date]
test_df = df[df['Date'] == split_date]

X_train = train_df[FEATURES]
y_train = train_df[TARGET]
X_test = test_df[FEATURES]
y_test = test_df[TARGET]

In [ ]:
model = 